In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from _entropy import calculate_entropy
from  _gini import calculate_gini_impurity

In [2]:
cinema_tennis = pd.read_csv('cinema_shopping_tenis_decision.csv')
cinema_tennis

,Weather,Parents,Money,Decision
0,Sunny,Yes,Rich,Cinema
1,Sunny,No,Rich,Tennis
2,Windy,Yes,Rich,Cinema
3,Rainy,Yes,Poor,Cinema
4,Rainy,No,Rich,Stay In
5,Rainy,Yes,Poor,Cinema
6,Windy,No,Poor,Cinema
7,Windy,No,Rich,Shopping
8,Windy,Yes,Rich,Cinema
9,Sunny,No,Rich,Tennis


In [3]:
gpt_house = pd.read_csv('gpt_house_category.csv')
gpt_house

,Size (sq. ft.),Number of Bedrooms,House Category
0,1500,3,Small House
1,1800,4,Medium House
2,2200,3,Small House
3,1300,2,Small House
4,2500,5,Large House
...,...,...,...
95,1600,5,Large House
96,1400,5,Large House
97,2700,2,Small House
98,2300,3,Small House


In [187]:
import pandas as pd

def split_based_on_gini_impurity(data, target, used_features=None):
    print("#############Split#################")
    lowest_impurity = calculate_gini_impurity(data, target)

    min_key, min_value = min(lowest_impurity.items(), key=lambda x: x[1])
    # print(lowest_impurity)
    # print(f"Next split based on feature: {min_key}, Gini Impurity: {min_value}")
    unique_values = data[min_key].unique()

    data_splits = {value: data[data[min_key] == value] for value in unique_values}

    splits = []
    
    for i, split_df in enumerate(data_splits.values(), 1):
        unique_values = split_df[target].unique()
        if len(unique_values) > 1:
            subtree_splits = split_based_on_gini_impurity(split_df, target, used_features)
            splits.extend(subtree_splits)
        else:
            # display(split_df.values)
            key_str = "-".join(str(val) for val in unique_values.tolist())
            split_dict = {'feature': min_key, 'value': split_df[min_key].values[0], 'prediction': unique_values[0]}
            splits.append(split_dict)
    
    return splits

decision_tree_splits = split_based_on_gini_impurity(data=cinema_tennis, target='Decision')

# Convert the splits to a DataFrame
decision_tree_df = pd.DataFrame(decision_tree_splits)

decision_tree_df

#############Split#################
#############Split#################
#############Split#################


,feature,value,prediction
0,Parents,Yes,Cinema
1,Weather,Sunny,Tennis
2,Weather,Rainy,Stay In
3,Money,Poor,Cinema
4,Money,Rich,Shopping


In [90]:
from bigtree import Node

a = Node("a", age=90)
b = Node("b", age=65, parent=a)
c = Node("c", age=60, parent=a)

a.children
# (Node(/a/b, age=65), Node(/a/c, age=60))

a.depth, b.depth
# # (1, 2)

a.max_depth
# # 2

a.show(attr_list=["age"])

a [age=90]
├── b [age=65]
└── c [age=60]


In [91]:
from bigtree import list_to_tree

path_list = ["a/b/d", "a/b/e", "a/c"]

root = list_to_tree(path_list)

root.show()

a
├── b
│   ├── d
│   └── e
└── c


In [94]:
from bigtree import dict_to_tree

path_dict = {
    "a": {"age": 90},
    "a/b": {"age": 65},
    "a/c": {"age": 60},
    "a/b/d": {"age": 40},
    "a/b/e": {"age": 35},
}
root = dict_to_tree(path_dict)

root.show(attr_list=["age"])

a [age=90]
├── b [age=65]
│   ├── d [age=40]
│   └── e [age=35]
└── c [age=60]


In [95]:
import pandas as pd
from bigtree import dataframe_to_tree

path_data = pd.DataFrame(
    [
        ["a", 90],
        ["a/b", 65],
        ["a/c", 60],
        ["a/b/d", 40],
        ["a/b/e", 35],
    ],
    columns=["PATH", "age"],
)
root = dataframe_to_tree(path_data)

root.show(attr_list=["age"])

a [age=90]
├── b [age=65]
│   ├── d [age=40]
│   └── e [age=35]
└── c [age=60]


In [98]:
from bigtree import Node, find, findall

root = Node("a", age=90)
b = Node("b", age=65, parent=root)
c = Node("c", age=60, parent=root)
d = Node("d", age=40, parent=b)

root.show(attr_list=["age"])

find(root, lambda node: node.age == 65)
# Node(/a/b, age=65)

findall(root, lambda node: node.age >= 60)
# (Node(/a, age=90), Node(/a/b, age=65), Node(/a/c, age=60))

a [age=90]
├── b [age=65]
│   └── d [age=40]
└── c [age=60]


(Node(/a, age=90), Node(/a/b, age=65), Node(/a/c, age=60))